## CNN sentence classifier - Hyperparameter tuning with reduced sentence length of 50 tokens.


In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 50


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 7.05 ms, sys: 15.8 ms, total: 22.8 ms
Wall time: 22 ms


## Tokenize the training data set and build a vocabulary from it


In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, x_val_ids, x_test_ids, tokenizer = tokenize_sentences(x_train, x_val, np.array([]), max_len)


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

CPU times: user 31.2 s, sys: 99.3 ms, total: 31.3 s
Wall time: 31.1 s


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [13]:
%%time
# this is very quick, so we can use grid search on these models.

run_logfile = "runs2ballen50.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models  (Removed the filter sizes 64 that exceed the size of the shorter sequence)
for k_sizes in [ [8,16], [16,32], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96, 96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal50",
                      model_dir        = "../models/CNNBal50",
                      max_len          = max_len)
            
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 4s 8ms/step - loss: 0.6859 - accuracy: 0.5957 - val_loss: 0.5588 - val_accuracy: 0.7315
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-005216/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4560 - accuracy: 0.7842 - val_loss: 0.4139 - val_accuracy: 0.8183
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-005216/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2908 - accuracy: 0.8816 - val_loss: 0.3055 - val_accuracy: 0.8834
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-005216/assets
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1746 - accuracy: 0.9367 - val_loss: 0.2323 - val_accuracy: 0.9155
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-005216/assets
Epoch 5/20
288/288 [=====

288/288 [==============================] - 1s 5ms/step - loss: 0.1027 - accuracy: 0.9642 - val_loss: 0.3234 - val_accuracy: 0.8983
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0679 - accuracy: 0.9788 - val_loss: 0.1588 - val_accuracy: 0.9539
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-005327/assets
Epoch 7/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0554 - accuracy: 0.9802 - val_loss: 0.2277 - val_accuracy: 0.9403
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0398 - accuracy: 0.9882 - val_loss: 0.2666 - val_accuracy: 0.9358
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0353 - accuracy: 0.9880 - val_loss: 0.2728 - val_accuracy: 0.9363
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0243 - accuracy: 0.9923 - val_loss: 0.2071 - val_accuracy: 0.9525
Epoch 11/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0212 -

288/288 [==============================] - 1s 5ms/step - loss: 0.0185 - accuracy: 0.9935 - val_loss: 0.4353 - val_accuracy: 0.9118
Epoch 13/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0150 - accuracy: 0.9943 - val_loss: 0.2803 - val_accuracy: 0.9458
Epoch 14/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0092 - accuracy: 0.9967 - val_loss: 0.5802 - val_accuracy: 0.9019
Epoch 15/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0121 - accuracy: 0.9961 - val_loss: 0.3042 - val_accuracy: 0.9399
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0099 - accuracy: 0.9977 - val_loss: 0.5896 - val_accuracy: 0.9037
Epoch 17/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0100 - accuracy: 0.9973 - val_loss: 0.3962 - val_accuracy: 0.9367
Epoch 18/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0050 - accuracy: 0.9984 - val_loss: 0.4850 - val_accuracy: 0.9227
Epoch 19/20

288/288 [==============================] - 1s 5ms/step - loss: 0.0106 - accuracy: 0.9965 - val_loss: 0.3293 - val_accuracy: 0.9376
Epoch 19/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0117 - accuracy: 0.9967 - val_loss: 0.3256 - val_accuracy: 0.9367
Epoch 20/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0133 - accuracy: 0.9959 - val_loss: 0.3747 - val_accuracy: 0.9381



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.6251 - accuracy: 0.6491 - val_loss: 0.4866 - val_accuracy: 0.7717
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-005619/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4053 - accuracy: 0.8156 - val_loss: 0.4089 - val_accuracy: 0.8237
INFO:tensorflow:Assets written to: .

Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1230 - accuracy: 0.9599 - val_loss: 0.2112 - val_accuracy: 0.9277
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-005728/assets
Epoch 5/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0701 - accuracy: 0.9789 - val_loss: 0.3592 - val_accuracy: 0.8892
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0410 - accuracy: 0.9883 - val_loss: 0.4073 - val_accuracy: 0.8933
Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0316 - accuracy: 0.9901 - val_loss: 0.1998 - val_accuracy: 0.9485
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-005728/assets
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0317 - accuracy: 0.9893 - val_loss: 0.3538 - val_accuracy: 0.9200
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0186 - accuracy: 0.9949 - val_loss: 0.4523 - val_accuracy: 0.905

...

Epoch 2/20
288/288 [==============================] - 2s 6ms/step - loss: 0.3614 - accuracy: 0.8426 - val_loss: 0.2485 - val_accuracy: 0.8992
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-015611/assets
Epoch 3/20
288/288 [==============================] - 2s 6ms/step - loss: 0.1976 - accuracy: 0.9255 - val_loss: 0.2741 - val_accuracy: 0.8870
Epoch 4/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0995 - accuracy: 0.9678 - val_loss: 0.2274 - val_accuracy: 0.9236
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-015611/assets
Epoch 5/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0506 - accuracy: 0.9869 - val_loss: 0.2361 - val_accuracy: 0.9290
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-015611/assets
Epoch 6/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0261 - accuracy: 0.9921 - val_loss: 0.3774 - val_accuracy: 0.9073
Epoch 7/20
288/288 [==============================] - 2s 6ms/step

INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-015734/assets
Epoch 7/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0334 - accuracy: 0.9889 - val_loss: 0.2175 - val_accuracy: 0.9480
Epoch 8/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0179 - accuracy: 0.9937 - val_loss: 0.1784 - val_accuracy: 0.9580
Epoch 9/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0277 - accuracy: 0.9914 - val_loss: 0.2581 - val_accuracy: 0.9372
Epoch 10/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0170 - accuracy: 0.9938 - val_loss: 0.4180 - val_accuracy: 0.9118
Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0095 - accuracy: 0.9978 - val_loss: 0.2204 - val_accuracy: 0.9548
Epoch 12/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0146 - accuracy: 0.9958 - val_loss: 0.4227 - val_accuracy: 0.9213
Epoch 13/20
288/288 [==============================] - 2s 7ms/step - l

288/288 [==============================] - 2s 6ms/step - loss: 0.0057 - accuracy: 0.9980 - val_loss: 0.7555 - val_accuracy: 0.8861
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0096 - accuracy: 0.9973 - val_loss: 0.2662 - val_accuracy: 0.9525
Epoch 16/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0021 - accuracy: 0.9993 - val_loss: 0.4629 - val_accuracy: 0.9231
Epoch 17/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0028 - accuracy: 0.9990 - val_loss: 0.5827 - val_accuracy: 0.9191
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0036 - accuracy: 0.9989 - val_loss: 0.3574 - val_accuracy: 0.9367
Epoch 19/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0057 - accuracy: 0.9976 - val_loss: 0.3597 - val_accuracy: 0.9412
Epoch 20/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0057 - accuracy: 0.9974 - val_loss: 0.5340 - val_accuracy: 0.9227



########

Adding Convolution: Kernel Size: 4, Filter Count: 8
Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Adding Convolution: Kernel Size: 32, Filter Count: 64
Epoch 1/20
288/288 [==============================] - 3s 8ms/step - loss: 0.6393 - accuracy: 0.6344 - val_loss: 0.4908 - val_accuracy: 0.7776
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-020110/assets
Epoch 2/20
288/288 [==============================] - 2s 6ms/step - loss: 0.3705 - accuracy: 0.8362 - val_loss: 0.4419 - val_accuracy: 0.8142
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-020110/assets
Epoch 3/20
288/288 [==============================] - 2s 6ms/step - loss: 0.2206 - accuracy: 0.9106 - val_loss: 0.1879 - val_accuracy: 0.9331
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-020110/assets
Epoch 4/20
288/288 [==============================] - 2s 6ms/step - loss: 0.1169 - accuracy: 0.9585 - val_loss: 0.1870 - val_accuracy: 0

288/288 [==============================] - 2s 6ms/step - loss: 0.1083 - accuracy: 0.9626 - val_loss: 0.2375 - val_accuracy: 0.9137
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-020236/assets
Epoch 5/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0566 - accuracy: 0.9825 - val_loss: 0.4565 - val_accuracy: 0.8734
Epoch 6/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0403 - accuracy: 0.9884 - val_loss: 0.1689 - val_accuracy: 0.9476
INFO:tensorflow:Assets written to: ../models/CNNBal50/210409-020236/assets
Epoch 7/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0272 - accuracy: 0.9916 - val_loss: 0.2255 - val_accuracy: 0.9430
Epoch 8/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0183 - accuracy: 0.9943 - val_loss: 0.5667 - val_accuracy: 0.8897
Epoch 9/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0278 - accuracy: 0.9903 - val_loss: 0.3947 - val_accuracy: 0.9195
Epoch 10/





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/